In [78]:
import numpy as np
import joblib
import pandas as pd
from pybit.unified_trading import HTTP
import time
# подгружаем новые свежие свечи и добавляем уже к имеющимся
df = pd.read_csv('./history_btc777.csv')
df['startTime'] = pd.to_datetime(df['startTime'])
df.set_index('startTime', inplace=True)

last_timestamp = df.index[-1]

session = HTTP(
    demo=True,
    api_key="",
    api_secret=""
)

current_time = int(time.time() * 1000)
chunk_size = 200
interval = 1
symbol = "BTCUSDT"
start_time = last_timestamp.timestamp() * 1000 # начало новых свечей
end_time = current_time # получаем свечи до нынешнего момента

def fetch_all_data(session, symbol, interval, start_time, end_time, chunk_size):
    all_data = []
    current_start = start_time

    while current_start < end_time:
        current_end = min(current_start + chunk_size * interval * 60 * 1000, end_time)  # Ограничиваем диапазон запроса

        response = session.get_kline(
            category="linear",
            symbol=symbol,
            interval=interval,
            start=current_start,
            end=current_end
        )

        if response['retCode'] == 0:
            all_data.extend(response['result']['list'])
            current_start = current_end + 1
        else:
            print("Ошибка в ответе API:", response['retMsg'])
            break

    return all_data

new_data = fetch_all_data(session, symbol, interval, start_time, end_time, chunk_size)

if new_data:
    new_df = pd.DataFrame(new_data, columns=["startTime", "openPrice", "highPrice", "lowPrice", "closePrice", "volume", "turnover"])
    new_df['startTime'] = pd.to_datetime(new_df['startTime'], unit='ms')
    new_df.set_index('startTime', inplace=True)
    new_df = new_df.astype('float64')

    df = pd.concat([df, new_df])
    df.sort_values(by='startTime', inplace=True)

    df.to_csv('./updated_history_btc.csv')
else:
    print("Нет новых данных для добавления.")
df

In [1]:
import pandas as pd
import numpy as np
df = pd.read_csv('./updated_history_btc.csv')
df.set_index('startTime', inplace=True)
df=df.drop(['Unnamed: 0','category','symbol'],axis=1)
df = df.rename(columns={'startTime': 'date'})
df = df[~df.index.duplicated(keep='last')]
df=df.tail(12000)
df.to_csv('./updated_history_btc_work.csv')
df

In [2]:
def create_target(df, atr_period=17, atr_threshold=60, price_increase_threshold=60, intervals=[3,4,5,6,7,8]):  
    
    for interval in intervals:
        df[f'price_increase_{interval}'] = df['closePrice'].shift(-interval) - df['closePrice']
    
    condition_price_increase = df[[f'price_increase_{interval}' for interval in intervals]].ge(price_increase_threshold).any(axis=1)
    
    df['signal'] = np.where(condition_price_increase, 1.0, 0.0)
    
    df = df.drop(columns=[f'price_increase_{interval}' for interval in intervals])
    
    return df

df = create_target(df)

df['signal'].value_counts()
df

In [3]:
df=df.tail(3000)

In [4]:
# import plotly.graph_objects as go
# import pandas as pd

# # Создание графика свечей
# fig = go.Figure(data=[go.Candlestick(x=df.index,
#                                      open=df['openPrice'],
#                                      high=df['highPrice'],
#                                      low=df['lowPrice'],
#                                      close=df['closePrice'])])

# # Добавление линии Close Price
# # fig.add_trace(go.Scatter(x=df.index, y=df['closePrice'], mode='lines', name='Close Price', line=dict(color='blue')))

# # Добавление линий скользящих средних (SMA1 и SMA2)
# # fig.add_trace(go.Scatter(x=df.index, y=df['SMA1'], mode='lines', name='SMA1 (10-дневная)', line=dict(dash='dash', color='orange')))
# # fig.add_trace(go.Scatter(x=df.index, y=df['SMA2'], mode='lines', name='SMA2 (60-дневная)', line=dict(dash='dash', color='purple')))

# # Добавление сигналов покупки и продажи
# buy_signals = df[df['signal'] == 1.0]
# sell_signals = df[df['signal'] == 0.0]

# fig.add_trace(go.Scatter(x=buy_signals.index, y=buy_signals['closePrice'], mode='markers', name='Buy Signal', marker=dict(symbol='triangle-up', color='yellow', size=5)))
# fig.add_trace(go.Scatter(x=sell_signals.index, y=sell_signals['closePrice'], mode='markers', name='Sell Signal', marker=dict(symbol='triangle-down', color='purple', size=5)))

# # Настройки графика
# fig.update_layout(
#     title='График свечей BTC/USDT с линиями SMA и сигналами',
#     xaxis_title='Дата',
#     yaxis_title='Цена',
#     xaxis_rangeslider_visible=True,
#     template='plotly_dark'
# )


# # Отображение графика
# fig.show()

In [129]:
''' Technical Indicators '''

#Calculation of moving average
def adaptive_moving_average(series, n):
    ema_fast = series.ewm(span=2).mean()
    ema_slow = series.ewm(span=n).mean()
    abs_diff = np.abs(ema_fast - ema_slow)
    volatility = abs_diff.rolling(n).mean()
    sc = 2 / (volatility + 1)
    ama = ema_fast + sc * (series - ema_fast)
    return ama

def atr(df, n):
    # Расчет True Range
    df['prev_close'] = df['closePrice'].shift(1)
    tr = pd.concat([
        df['highPrice'] - df['lowPrice'],
        abs(df['highPrice'] - df['prev_close']),
        abs(df['lowPrice'] - df['prev_close'])
    ], axis=1).max(axis=1)

    # Расчет ATR
    atr = tr.rolling(window=n, min_periods=1).mean()

    return atr

def ma(df, n):
    return pd.Series(df['closePrice'].rolling(n, min_periods=n).mean(), name='MA_' + str(n))

# exponentially weighted moving average 
def ema(df, n):
    return pd.Series(df['closePrice'].ewm(span=n,min_periods=n).mean(), name='EMA_' + str(n))

#Calculation of price momentum
def mom(df, n):     
    return pd.Series(df.diff(n), name='Momentum_' + str(n))  

# rate of change
def roc(df, n):  # Показывает на сколько процентов цена увеличилась/уменьшилась
    M = df.diff(n - 1) ; N = df.shift(n - 1)  
    return pd.Series(((M / N) * 100), name = 'ROC_' + str(n)) 

# relative strength index
def rsi(df, period): # значение перекупленности или перепроданности от 0 до 100
    delta = df.diff().dropna()
    u = delta * 0; d = u.copy()
    u[delta > 0] = delta[delta > 0]; d[delta < 0] = -delta[delta < 0]
    u[u.index[period-1]] = np.mean( u[:period] ) #first value is sum of avg gains
    u = u.drop(u.index[:(period-1)])
    d[d.index[period-1]] = np.mean( d[:period] ) #first value is sum of avg losses
    d = d.drop(d.index[:(period-1)])
    rs = u.ewm(com=period-1, adjust=False).mean() / d.ewm(com=period-1, adjust=False).mean()
    return 100 - 100 / (1 + rs)

# stochastic oscillators slow & fast
def sto(close, low, high, n,id): 
    stok = ((close - low.rolling(n).min()) / (high.rolling(n).max() - low.rolling(n).min())) * 100
    if(id == 0):
        return stok
    else:
        return stok.rolling(3).mean()
    
def macd(df):
    exp1 = df['closePrice'].ewm(span=12, adjust=False).mean()
    exp2 = df['closePrice'].ewm(span=26, adjust=False).mean()
    macd_line = exp1 - exp2
    signal_line = macd_line.ewm(span=9, adjust=False).mean()
    return macd_line, signal_line    
# %K близко к 0 указывает на то, что текущая цена ближе к минимальной цене за период, 
# что может сигнализировать о перепроданности рынка.
# %K близко к 100 указывает на то, что текущая цена ближе к максимальной цене за период,
# что может сигнализировать о перекупленности рынка.

def adx(df, n):
    df['TR'] = atr(df, 1)  # True Range используется в расчете ADX
    df['+DM'] = df['highPrice'].diff()
    df['-DM'] = df['lowPrice'].diff()
    
    df['+DM'] = np.where((df['+DM'] > df['-DM']) & (df['+DM'] > 0), df['+DM'], 0.0)
    df['-DM'] = np.where((df['-DM'] > df['+DM']) & (df['-DM'] > 0), df['-DM'], 0.0)
    
    df['+DI'] = 100 * (df['+DM'].ewm(alpha=1/n, min_periods=n).mean() / df['TR'].ewm(alpha=1/n, min_periods=n).mean())
    df['-DI'] = 100 * (df['-DM'].ewm(alpha=1/n, min_periods=n).mean() / df['TR'].ewm(alpha=1/n, min_periods=n).mean())
    
    df['DX'] = 100 * np.abs((df['+DI'] - df['-DI']) / (df['+DI'] + df['-DI']))
    adx = df['DX'].ewm(alpha=1/n, min_periods=n).mean()
    
    return adx


In [130]:
def tech_indi(ldf,tr_id=True):
#     df = df.drop(columns=['openPrice','highPrice','lowPrice','closePrice','turnover','volume','MA63','EMA10','%K10'])

    ldf['ATR14'] = atr(ldf, 41)
    
    lst_ATR = ['ATR14']
    
#     ldf['ADX200'] = adx(ldf, 20)
#     lst_ADX = ['ADX200']

    
    ldf['AMA'] = adaptive_moving_average(ldf['closePrice'], 10)
    ldf['AMA'] = adaptive_moving_average(ldf['closePrice'], 15)
    lst_AMA = ['AMA']

    ''' Moving Average '''
#     ldf['MA21'] = ma(ldf,10)
    ldf['MA63'] = ma(ldf, 30)
    ldf['MA63'] = ma(ldf, 35)

    ldf['MA63'] = ma(ldf, 40)
    ldf['MA63'] = ma(ldf, 45)

    ldf['MA63'] = ma(ldf, 60)
    ldf['MA63'] = ma(ldf, 252)

    lst_MA = ['MA21','MA63','MA252']

    ''' Exponentially Weighted Moving Average '''
#     ldf['EMA10'] = ema(ldf, 10)
    ldf['EMA30'] = ema(ldf, 30)
    ldf['EMA200'] = ema(ldf, 200)
    ldf['EMA200'] = ema(ldf, 300)
    lst_EMA = ['EMA10','EMA30','EMA200']

    ''' Momentum '''
    ldf['MOM10'] = mom(ldf['closePrice'], 10)
    ldf['MOM30'] = mom(ldf['closePrice'], 15)

    ldf['MOM30'] = mom(ldf['closePrice'], 20)
    ldf['MOM30'] = mom(ldf['closePrice'], 25)

    ldf['MOM30'] = mom(ldf['closePrice'], 30)
    ldf['MOM30'] = mom(ldf['closePrice'], 35)

    ldf['MOM30'] = mom(ldf['closePrice'], 40)
    ldf['MOM30'] = mom(ldf['closePrice'], 45)

    lst_MOM = ['MOM10','MOM30']

#     ''' Relative Strength Index '''
#     ldf['RSI10'] = rsi(ldf['closePrice'], 10)
#     ldf['RSI30'] = rsi(ldf['closePrice'], 30)
#     ldf['RSI200'] = rsi(ldf['closePrice'], 200)
#     lst_RSI = ['RSI10','RSI30','RSI200']

    ''' Slow Stochastic Oscillators '''
#     ldf['%K10'] = sto(ldf['closePrice'], ldf['lowPrice'], ldf['highPrice'],5,0)
    ldf['%K30'] = sto(ldf['closePrice'], ldf['lowPrice'], ldf['highPrice'],10,0)
    ldf['%K200'] = sto(ldf['closePrice'], ldf['lowPrice'], ldf['highPrice'], 20,0)
    ldf['%K200'] = sto(ldf['closePrice'], ldf['lowPrice'], ldf['highPrice'], 30,0)
    lst_pK = ['%K10','%K30','%K200']

    ''' Fast Stochastic Oscillators '''
#     ldf['%D10'] = sto(ldf['closePrice'], ldf['lowPrice'], ldf['highPrice'], 10,1)
    ldf['%D30'] = sto(ldf['closePrice'], ldf['lowPrice'], ldf['highPrice'], 30,1)
    ldf['%D200'] = sto(ldf['closePrice'], ldf['lowPrice'], ldf['highPrice'], 250,1)
    ldf['%D200'] = sto(ldf['closePrice'], ldf['lowPrice'], ldf['highPrice'], 300,1)
    
    lst_pD = ['%D10','%D30','%D200']
    
    ldf['MACD'], ldf['Signal_Line'] = macd(ldf)


In [5]:
tech_indi(df)

In [132]:
df = df.dropna()

In [133]:
NaN_values = (df.isnull().sum() / len(df) * 100).sort_values(ascending = False)
NaN_values

openPrice      0.0
EMA30          0.0
MACD           0.0
%D200          0.0
%D30           0.0
%K200          0.0
%K30           0.0
MOM30          0.0
MOM10          0.0
EMA200         0.0
MA63           0.0
highPrice      0.0
AMA            0.0
ATR14          0.0
prev_close     0.0
signal         0.0
turnover       0.0
volume         0.0
closePrice     0.0
lowPrice       0.0
Signal_Line    0.0
dtype: float64

In [134]:
# df = df.drop(columns=['openPrice','highPrice','lowPrice','closePrice','MA63','EMA10','%K10'])
# df = df.drop(columns=['openPrice','highPrice','lowPrice','closePrice','turnover','volume'])
# df = df.drop(columns=['openPrice','highPrice','lowPrice','closePrice','turnover','volume'])

In [135]:
from sklearn.model_selection import train_test_split
test_size = 500
eval_size = 500

test_df = df.iloc[-test_size:]
eval_df = df.iloc[-(eval_size + test_size):-test_size]
tr_df = df.iloc[:-(eval_size + test_size)]

y_test = test_df['signal']
X_test = test_df.loc[:, test_df.columns != 'signal']

y_eval = eval_df['signal']
X_eval = eval_df.loc[:, eval_df.columns != 'signal']

y_train = tr_df['signal']
X_train = tr_df.loc[:, tr_df.columns != 'signal']

# eval_size = 20000  

# eval_df = df.iloc[-eval_size:]  
# train_df = df.iloc[:-eval_size]

# y_train = train_df['signal']
# X_train = train_df.drop(columns=['signal'])

# y_eval = eval_df['signal']
# X_eval = eval_df.drop(columns=['signal'])

In [136]:
import numpy as np
import tensorflow as tf
from tensorflow.keras import layers, models, Sequential
from tensorflow.keras.callbacks import EarlyStopping
from sklearn.metrics import classification_report, f1_score
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from tensorflow.keras.regularizers import l2
from bayes_opt import BayesianOptimization
import random
from tensorflow.keras.callbacks import EarlyStopping, Callback

np.random.seed(42)
tf.random.set_seed(42)
random.seed(42)

X_train_cnn = np.expand_dims(X_train, axis=-1)
X_eval_cnn = np.expand_dims(X_eval, axis=-1)

scaler = StandardScaler()
X_train_cnn = scaler.fit_transform(X_train_cnn.reshape(-1, X_train_cnn.shape[-2])).reshape(X_train_cnn.shape)
X_eval_cnn = scaler.transform(X_eval_cnn.reshape(-1, X_eval_cnn.shape[-2])).reshape(X_eval_cnn.shape)

class BestMetrics(Callback):
    def __init__(self, validation_data, thresholds):
        super(BestMetrics, self).__init__()
        self.validation_data = validation_data
        self.thresholds = thresholds
        self.best_val_loss = np.Inf
        self.best_f1 = 0
        self.best_threshold = 0

    def on_epoch_end(self, epoch, logs=None):
        current_val_loss = logs.get('val_loss')
        
        if current_val_loss < self.best_val_loss:
            self.best_val_loss = current_val_loss
        
        X_val, y_val = self.validation_data
        y_pred_prob = self.model.predict(X_val)
        
        best_f1_epoch = 0
        best_threshold_epoch = 0

        for threshold in self.thresholds:
            y_pred = (y_pred_prob > threshold).astype(int)
            f1 = f1_score(y_val, y_pred, average='micro')
            print(f'Epoch {epoch+1} - Threshold {threshold} - Precision: {f1}')
            
            if f1 > best_f1_epoch:
                best_f1_epoch = f1
                best_threshold_epoch = threshold

        print(f'Epoch {epoch+1} - Best Threshold: {best_threshold_epoch} - Best Precision: {best_f1_epoch}')

        if best_f1_epoch > self.best_f1:
            self.best_f1 = best_f1_epoch
            self.best_threshold = best_threshold_epoch
            
# Функция для компиляции модели
def compile_model(learning_rate):
    # model = Sequential()
    # model.add(layers.Conv1D(filters=32, kernel_size=3, activation='relu', input_shape=(X_train_cnn.shape[1], 1), kernel_regularizer=l2(0.001)))
    # model.add(layers.MaxPooling1D(pool_size=2))
    # model.add(layers.Flatten())
    # model.add(layers.Dense(64, activation='relu', kernel_regularizer=l2(0.001)))
    # model.add(layers.Dropout(0.2))
    # model.add(layers.Dense(1, activation='sigmoid', kernel_regularizer=l2(0.001)))
    model = Sequential()
    model.add(layers.LSTM(64, return_sequences=True, input_shape=(X_train_cnn.shape[1], 1), kernel_regularizer=l2(0.001)))
    model.add(layers.LSTM(128))
    model.add(layers.Dense(64, activation='relu', kernel_regularizer=l2(0.001)))
    model.add(layers.Dropout(0.2))
    model.add(layers.Dense(1, activation='sigmoid', kernel_regularizer=l2(0.001)))

    optimizer = tf.keras.optimizers.Adam(learning_rate=learning_rate)
    model.compile(optimizer=optimizer, loss='binary_crossentropy', metrics=['Precision'])
    
    return model

# Функция для обучения модели и оценки F1-score
def train_evaluate(batch_size, learning_rate):
    batch_size = int(batch_size)
    
    model = compile_model(learning_rate)

    early_stopping = EarlyStopping(monitor='val_loss', patience=2, mode='min', verbose=1, restore_best_weights=True)
    f1_callback = BestMetrics(validation_data=(X_eval_cnn, y_eval), thresholds=[0.3, 0.4, 0.5, 0.6, 0.7])

    model.fit(X_train_cnn, y_train, 
              epochs=100, 
              batch_size=batch_size, 
              validation_data=(X_eval_cnn, y_eval), 
              callbacks=[f1_callback, early_stopping],
              verbose=0)  # Устанавливаем verbose на 0, чтобы не печатать прогресс по эпохам

    # Возвращаем лучший F1 score на валидации
    return f1_callback.best_f1

# # Определение пространства поиска для байесовской оптимизации
# pbounds = {'batch_size': (32, 128), 'learning_rate': (0.001, 0.01)}

# # Инициализация байесовской оптимизации
# optimizer = BayesianOptimization(
#     f=train_evaluate,
#     pbounds=pbounds,
#     random_state=42,
# )

# # Запуск байесовской оптимизации
# optimizer.maximize(
#     init_points=5,
#     n_iter=15,
# )

# # Выводим лучшие результаты
# print(optimizer.max)


In [4]:
import numpy as np
import tensorflow as tf
import random
from sklearn.preprocessing import StandardScaler
from tensorflow.keras.callbacks import EarlyStopping

physical_devices = tf.config.list_physical_devices('GPU')
if physical_devices:
    tf.config.experimental.set_visible_devices(physical_devices[0], 'GPU')
    print("используем GPU")
    
np.random.seed(42)
tf.random.set_seed(42)
random.seed(42)

X_train_cnn = np.expand_dims(X_train, axis=-1)
X_eval_cnn = np.expand_dims(X_eval, axis=-1)

scaler = StandardScaler()
X_train_cnn = scaler.fit_transform(X_train_cnn.reshape(-1, X_train_cnn.shape[-2])).reshape(X_train_cnn.shape)
X_eval_cnn = scaler.transform(X_eval_cnn.reshape(-1, X_eval_cnn.shape[-2])).reshape(X_eval_cnn.shape)

early_stopping = EarlyStopping(monitor='val_loss', patience=2, mode='min', verbose=1, restore_best_weights=True)
f1_callback = BestMetrics(validation_data=(X_eval_cnn, y_eval), thresholds=[0.3, 0.4, 0.5, 0.6, 0.7])

# model = models.Sequential()

# model.add(layers.Conv1D(filters=64, kernel_size=3, activation='relu', input_shape=(X_train_cnn.shape[1], 1)))
# model.add(layers.MaxPooling1D(pool_size=2))
# model.add(layers.Conv1D(filters=128, kernel_size=3, activation='relu'))
# model.add(layers.MaxPooling1D(pool_size=2))
# model.add(layers.Conv1D(filters=256, kernel_size=3, activation='relu'))
# model.add(layers.Flatten())
# model.add(layers.Dense(128, activation='relu'))
# model.add(layers.Dense(1, activation='sigmoid'))

model = tf.keras.Sequential()
model.add(tf.keras.layers.LSTM(64, return_sequences=True, input_shape=(X_train_cnn.shape[1], 1)))
model.add(tf.keras.layers.LSTM(128, return_sequences=True))
model.add(tf.keras.layers.LSTM(256))
model.add(tf.keras.layers.Dense(128, activation='relu'))
model.add(tf.keras.layers.Dense(1, activation='sigmoid'))

optimizer = tf.keras.optimizers.Adam(learning_rate=0.002)
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['Precision'])

history= model.fit(X_train_cnn, y_train,
                   epochs=1000,
                   batch_size=132,
                   validation_data=(X_eval_cnn, y_eval),
                   callbacks=[f1_callback, early_stopping])

physical_devices = tf.config.list_physical_devices('GPU')

print(f"Best Precision: {f1_callback.best_f1}")
print(f"Best Threshold: {f1_callback.best_threshold}")
print(f"Best val_loss: {f1_callback.best_val_loss}")

In [3]:
# import h5py
X_test_cnn = np.expand_dims(X_test, axis=-1)  # Добавляем канал
X_test_cnn = scaler.transform(X_test_cnn.reshape(-1, X_test_cnn.shape[-2])).reshape(X_test_cnn.shape)

y_pred = model.predict(X_test_cnn)
# y_pred = model.predict(X_eval_cnn)

y_pred = (y_pred > 0.74).astype(int)  # Порог 0.5 для бинарной классификации

print(classification_report(y_test, y_pred))
# print(classification_report(y_eval, y_pred))

# Сохранение модели
# model_filename = 'cnn_model.h5'
# model.save(model_filename)
# print(f"Модель сохранена в файл: {model_filename}")

In [145]:
import plotly.graph_objects as go
import pandas as pd

# Предполагается, что eval_df уже содержит предсказанные сигналы и данные для построения графика
# Если нет, добавьте их, как в предыдущем примере:
# eval_df['predicted_signal'] = y_pred
test_df['predicted_signal'] = y_pred

# Создание графика свечей
fig = go.Figure(data=[go.Candlestick(x=test_df.index,
                                     open=test_df['openPrice'],
                                     high=test_df['highPrice'],
                                     low=test_df['lowPrice'],
                                     close=test_df['closePrice'])])

# Добавление линии для фактических цен закрытaия
fig.add_trace(go.Scatter(x=test_df.index, y=test_df['closePrice'], mode='lines', name='Close Price', line=dict(color='blue')))

# Добавление сигналов покупки и продажи
buy_signals = test_df[test_df['predicted_signal'] == 1.0]
sell_signals = test_df[test_df['predicted_signal'] == 0.0]

fig.add_trace(go.Scatter(x=buy_signals.index, y=buy_signals['closePrice'], mode='markers', name='Buy Signal', marker=dict(symbol='triangle-up', color='yellow', size=5)))
fig.add_trace(go.Scatter(x=sell_signals.index, y=sell_signals['closePrice'], mode='markers', name='Sell Signal', marker=dict(symbol='triangle-down', color='purple', size=5)))

# Настройки графика
fig.update_layout(
    title='Фактические цены закрытия и предсказанные сигналы на покупку/продажу (свечи)',
    xaxis_title='Дата',
    yaxis_title='Цена закрытия',
    legend_title='Легенда',
    xaxis_rangeslider_visible=True,
    template='plotly_dark'
)

# Отображение графика
fig.show()


C:\Users\Waple\AppData\Local\Temp\ipykernel_12064\1310145631.py:7: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [75]:
# import plotly.graph_objects as go # 11- 1230 1350 0132
# import pandas as pd

# # Предполагается, что eval_df уже содержит предсказанные сигналы и данные для построения графика
# # Если нет, добавьте их, как в предыдущем примере:
# # eval_df['predicted_signal'] = y_pred
# eval_df['predicted_signal'] = y_pred

# # Создание графика свечей
# fig = go.Figure(data=[go.Candlestick(x=eval_df.index,
#                                      open=eval_df['openPrice'],
#                                      high=eval_df['highPrice'],
#                                      low=eval_df['lowPrice'],
#                                      close=eval_df['closePrice'])])

# # Добавление линии для фактических цен закрытия
# fig.add_trace(go.Scatter(x=eval_df.index, y=eval_df['closePrice'], mode='lines', name='Close Price', line=dict(color='blue')))

# # Добавление сигналов покупки и продажи
# buy_signals = eval_df[eval_df['predicted_signal'] == 1.0]
# sell_signals = eval_df[eval_df['predicted_signal'] == 0.0]

# fig.add_trace(go.Scatter(x=buy_signals.index, y=buy_signals['closePrice'], mode='markers', name='Buy Signal', marker=dict(symbol='triangle-up', color='yellow', size=5)))
# fig.add_trace(go.Scatter(x=sell_signals.index, y=sell_signals['closePrice'], mode='markers', name='Sell Signal', marker=dict(symbol='triangle-down', color='purple', size=5)))

# # Настройки графика
# fig.update_layout(
#     title='Фактические цены закрытия и предсказанные сигналы на покупку/продажу (свечи)',
#     xaxis_title='Дата',
#     yaxis_title='Цена закрытия',
#     legend_title='Легенда',
#     xaxis_rangeslider_visible=True,
#     template='plotly_dark'
# )

# # Отображение графика
# fig.show()


C:\Users\Waple\AppData\Local\Temp\ipykernel_12064\1631960169.py:7: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [197]:
import numpy as np
from xgboost import XGBClassifier
import joblib
import pandas as pd
from pybit.unified_trading import HTTP
import time
from IPython.display import display

df = pd.read_csv('./updated_history_btc_work.csv')
# df=df.drop(['Unnamed: 0','symbol','category'], axis=1)
df['startTime'] = pd.to_datetime(df['startTime'])
df.set_index('startTime', inplace=True)

last_timestamp = df.index[-1]

session = HTTP(
    demo=True,
    api_key="",
    api_secret=""
)

current_time = int(time.time() * 1000)

response = session.get_kline(
    category="linear",
    symbol="BTCUSDT",
    interval=1,
    start_time=int(last_timestamp.timestamp() * 1000),
    end_time=current_time
)

if response['retCode'] == 0:
    new_data = response['result']['list']

    new_df = pd.DataFrame(new_data, columns=["startTime", "openPrice", "highPrice", "lowPrice", "closePrice", "volume", "turnover"])
    new_df['startTime'] = pd.to_datetime(new_df['startTime'], unit='ms')
    new_df.set_index('startTime', inplace=True)
    new_df = new_df.astype('float64')

    df = pd.concat([df, new_df])
    df.sort_values(by='startTime', inplace=True)
    df.to_csv('./updated_history_btc_work.csv')
else:
    print(f"Ошибка получения данных: {response['retMsg']}")

C:\Users\Waple\AppData\Local\Temp\ipykernel_29768\2449148299.py:36: FutureWarning:

The behavior of 'to_datetime' with 'unit' when parsing strings is deprecated. In a future version, strings will be parsed as datetime strings, matching the behavior without a 'unit'. To retain the old behavior, explicitly cast ints or floats to numeric type before calling to_datetime.



In [198]:
import numpy as np
from xgboost import XGBClassifier
import joblib
import pandas as pd
from pybit.unified_trading import HTTP
import time

session = HTTP(
    api_key="",
    api_secret=""
)

df = pd.read_csv('./updated_history_btc_work.csv')
df.set_index('startTime', inplace=True)
df = df[~df.index.duplicated(keep='last')]
# df = df.drop(columns=['openPrice','highPrice','lowPrice','closePrice','volume','MA63','EMA10','%K10'])

df

,openPrice,highPrice,lowPrice,closePrice,volume,turnover
startTime,,,,,,
2024-07-30 08:36:00,66821.1,66821.1,66790.5,66790.6,10.702,7.149464e+05
2024-07-30 08:37:00,66790.6,66815.7,66790.6,66808.2,11.228,7.500986e+05
2024-07-30 08:38:00,66808.2,66815.0,66803.3,66814.9,7.795,5.207614e+05
2024-07-30 08:39:00,66814.9,66815.0,66791.9,66815.0,13.672,9.133660e+05
2024-07-30 08:40:00,66815.0,66839.8,66814.9,66835.1,39.379,2.631704e+06
...,...,...,...,...,...,...
2024-09-23 21:53:00,63308.5,63349.4,63308.4,63349.3,19.306,1.222528e+06
2024-09-23 21:54:00,63349.3,63351.3,63344.4,63351.3,14.717,9.322985e+05
2024-09-23 21:55:00,63351.3,63351.3,63336.5,63340.0,12.519,7.930088e+05


In [199]:
def place_order_buy(session, symbol, side, order_type, price, qty, take_profit_price, take_profit_price_limit,stopLoss,slLimitPrice):
    order_response = session.place_order(
        category="spot",
        symbol=symbol,
        side=side,
        orderType=order_type,
        marketUnit="baseCoin",
        price=price,
        timeInForce="FOK",
        qty=qty,
        takeProfit=str(take_profit_price),
        tpLimitPrice=str(take_profit_price_limit),
        tpTriggerBy='LastPrice',
        tpOrderType="Limit",
        ##
        stopLoss=str(stopLoss),
        slLimitPrice=str(slLimitPrice),
        slTriggerBy='LastPrice',
        slOrderType="Limit"
        ##
    )
    return order_response

def place_order_sell(session, symbol, side, order_type, price, qty):
    order_response= session.place_order(
            category="spot",
            symbol=symbol,
            side=side,
            orderType=order_type,
            marketUnit="baseCoin",
            price=price,
            timeInForce="FOK",
            qty=qty,
        )
    return order_response

In [1]:
import numpy as np
from xgboost import XGBClassifier
import joblib
import pandas as pd
from pybit.unified_trading import HTTP
import time
from datetime import datetime, timedelta
import threading
pd.set_option('display.max_columns', None)
# 400 и 329 попробовать
session = HTTP(
    demo=True,
    api_key="",
    api_secret=""
)

# xgb_model = joblib.load('xgb_modelnew1337.pkl')

last_purchase_time = None
min_interval_minutes = 5

import tensorflow as tf

# Загрузка модели
model_filename = 'cnn_model444.h5'
cnn_model = tf.keras.models.load_model(model_filename)

symbol = "BTCUSDT"


last_signal = 0
order_id = None
last_time = None

while True:
    try:
        current_time = pd.Timestamp.now()
        if current_time.second == 0:
            prev_minute_start_time = int((datetime.now() - timedelta(minutes=1)).timestamp() * 1000)
            print("Сейчас время:")
            print(time.ctime())
            print("Смотрим на пред. свечу:")
            print(pd.to_datetime(prev_minute_start_time, unit='ms'))

            response = session.get_kline(
                category="linear",
                symbol=symbol,
                interval=1,
                limit=1,
                start=prev_minute_start_time,
                end=prev_minute_start_time + 5000
            )
            print(response)

            if response['retCode'] == 0:
                all_data = response['result']['list']
                rows = [item for item in all_data]

                new_df = pd.DataFrame(rows, columns=["startTime", "openPrice", "highPrice", "lowPrice", "closePrice", "volume", "turnover"])

                new_df['startTime'] = pd.to_datetime(new_df['startTime'], unit='ms')
                new_df.set_index('startTime', inplace=True)
                new_df = new_df.astype('float64')

                if last_time is None or new_df.index[0] > last_time:
                    last_time = new_df.index[0]

#                     df_fake0 = df.tail(3000)
                    df_fake0 = df

                    df_fake1 = pd.concat([df_fake0, new_df])
                    tech_indi(df_fake1)
                    last_row_tail = df_fake1.tail(1)
#                     last_row_tail = last_row_tail.drop(columns=['openPrice','highPrice','lowPrice','closePrice','volume','MA63','EMA10','%K10'])

                    df = pd.concat([df, last_row_tail])
    
                    last_row_tail_cnn = np.expand_dims(last_row_tail, axis=-1)
                    last_row_tail_cnn = scaler.transform(last_row_tail_cnn.reshape(-1, last_row_tail_cnn.shape[-2])).reshape(last_row_tail_cnn.shape)

                    signal = cnn_model.predict(last_row_tail_cnn)
                    print("roow")
                    print(last_row_tail_cnn)
                    print(signal)

                    signal = (signal > 0.59).astype(int)
                    print(signal)    

                    print(display(df))
#                     signal = model_filename.predict(last_row_tail)
#                     print("предсказывает сигнаалл")
#                     print(signal[0])

                    ticker = session.get_tickers(category="spot", symbol="BTCUSDT")
                    price = float(ticker['result']['list'][0]['lastPrice'])
                    take_profit_price = price + 319
                    take_profit_price_limit = take_profit_price + 1
                    ##
                    stopLoss=price-180
                    slLimitPrice=stopLoss-1
                    ##
                    
                    response = session.get_order_history(
                    category="spot",
                    symbol="BTCUSDT",
                    orderId=order_id)
                    
                    history_order=session.get_order_history(
                    category="spot",
                    limit=1)
                    if history_order['result']['list'][0]['stopOrderType']=='tpslOrder' and history_order['result']['list'][0]['side']=='Sell'and history_order['result']['list'][0]['orderStatus']=='Filled':
                        last_signal = 0
                        ##
                    if history_order['result']['list'][0]['stopOrderType']=='BidirectionalTpslOrder' and history_order['result']['list'][0]['side']=='Sell'and history_order['result']['list'][0]['orderStatus']=='Filled':
                        last_signal = 0    
                        ##
#                     if signal[0, 0] == 1 and last_signal == 0:

#                         mytime = time.ctime()
#                         print(f"Покупка BTC в {mytime}")
#                         ##
#                         order_response = place_order_buy(session, symbol="BTCUSDT", side="Buy", order_type="Limit", price=price, qty=0.1, take_profit_price=take_profit_price, take_profit_price_limit=take_profit_price_limit,stopLoss=stopLoss,slLimitPrice=slLimitPrice)
#                         ##
#                         order_id = order_response['result']['orderId']
                        
                    if signal[0, 0] == 1 and last_signal == 0:
                        if last_purchase_time is None or (datetime.now() - last_purchase_time) >= timedelta(minutes=min_interval_minutes):
                            mytime = time.ctime()
                            print(f"Покупка BTC в {mytime}")

                            order_response = place_order_buy(session, symbol="BTCUSDT", side="Buy", order_type="Limit",price=price, qty=0.1, take_profit_price=take_profit_price,take_profit_price_limit=take_profit_price_limit, stopLoss=stopLoss, slLimitPrice=slLimitPrice)
                            order_id = order_response['result']['orderId']
                            last_purchase_time = datetime.now()  # Сохраняем вре    
                        while True:
                                try:
                                    response = session.get_order_history(
                                        category="spot",
                                        symbol="BTCUSDT",
                                        orderId=order_id
                                    )

                                    if response['retCode'] == 0:
                                        order_info = response['result']

                                        if order_info['list'][0]['orderStatus'] == "Filled":
                                            print(f"Ордер {order_id} выполнен!")
                                            last_signal = 1
                                            break
                                        elif order_info['list'][0]['orderStatus'] != "Filled":
                                            ticker = session.get_tickers(category="spot", symbol="BTCUSDT")
                                            price=float(ticker['result']['list'][0]['lastPrice'])
                                            price=price
                                            print(f"Новая цена {price}.")
                                            order_response = place_order_buy(session, symbol="BTCUSDT", side="Buy", order_type="Limit", price=price, qty=0.1, take_profit_price=take_profit_price, take_profit_price_limit=take_profit_price_limit,stopLoss=stopLoss,slLimitPrice=slLimitPrice)
                                            order_id = order_response['result']['orderId']

                                            print(f"Ордер был отменен.Создали новый {order_id}")
                                            continue
                                    else:
                                        print(f"Ошибка при проверке статуса ордера: {response['retMsg']}")
                                        break  
                                except Exception as e:
                                    print(f"Ошибка при проверке ордера: {e}")
                                time.sleep(0.1)  

                    elif signal == 0 and last_signal != 0:
                        mytime = time.ctime()
                        print(f"Продажа BTC в {mytime}")
                        order_response = place_order_sell(session, symbol="BTCUSDT", side="Sell", order_type="Limit", price=price, qty=0.1)
                        order_id = order_response['result']['orderId']
                        
                        while True:
                                try:
                                    response = session.get_order_history(
                                        category="spot",
                                        symbol="BTCUSDT",
                                        orderId=order_id
                                    )

                                    if response['retCode'] == 0:
                                        order_info = response['result']

                                        if order_info['list'][0]['orderStatus'] == "Filled":
                                            print(f"Ордер {order_id} выполнен!")
                                            last_signal = 0
                                            ##
                                            responseOpenOrders = session.get_open_orders(
                                            category="spot",
                                            symbol="BTCUSDT")

                                            if responseOpenOrders['retCode'] == 0:
                                                tp_orders = []
                                                for order in responseOpenOrders['result']['list']:
                                                    if order.get('stopOrderType') == 'tpslOrder' and order.get('side') == 'Sell':
                                                        tp_orders.append(order)

                                                for order in tp_orders:
                                                    order_id = order['orderId']
                                                    cancel_response = session.cancel_order(
                                                        category="spot",
                                                        symbol="BTCUSDT",
                                                        orderId=order_id
                                                    )
                                                    if cancel_response['retCode'] == 0:
                                                        print(f"Ордер {order_id} был успешно отменен.")
                                                    else:
                                                        print(f"Не удалось отменить ордер {order_id}. Код ошибки: {cancel_response['retCode']}")
                                            else:
                                                print(f"Не удалось получить открытые ордера. Код ошибки: {responseOpenOrders['retCode']}")
                                            ##
                                            last_signal = 0
                                            
                                            break
                                        elif order_info['list'][0]['orderStatus'] != "Filled":
                                            ticker = session.get_tickers(category="spot", symbol="BTCUSDT")
                                            price=float(ticker['result']['list'][0]['lastPrice'])
                                            price=price
                                            print(f"Новая цена {price}.")
                                            order_response = place_order_sell(session, symbol="BTCUSDT", side="Sell", order_type="Limit", price=price, qty=0.1)
                                            order_id = order_response['result']['orderId']

                                            print(f"Ордер был отменен.Создали новый {order_id}")
                                            continue
                                    else:
                                        print(f"Ошибка при проверке статуса ордера: {response['retMsg']}")
                                        break  
                                except Exception as e:
                                    print(f"Ошибка при проверке ордера: {e}")
                                time.sleep(0.1)  
                                
    except Exception as e:
        print(f"Произошла ошибка: {e}")

    time.sleep(0.5)